# imported libraries

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import re
import numpy as np  

# load specific datset

In [2]:
df = pd.read_csv("d:/dataset/dataset/spam.csv", encoding='latin1')
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


# check null values and remove null values

In [3]:
df.notnull().sum()
df = df.drop(['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'], axis=1)

# create column label

In [4]:
df.columns=['feedback','Review']
df

,feedback,Review
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


# extract value from a dataset in the form of features matrix[X] $ traget matrix[y]

In [5]:
X=df.Review
y=df.feedback


# data preprocessing

In [6]:
from nltk.stem import WordNetLemmatizer
wlen=WordNetLemmatizer()
sw=stopwords.words('english')
sw.remove('not')

def text_cleaning(doc):
        doc=doc.lower()
        doc=re.sub("[^a-z]"," ",doc)
        tokens=word_tokenize(doc)
        new_doc=[]
        for token in tokens:
            token = wlen.lemmatize(token)  
            if token in sw:
                pass
            else:
                 new_doc.append(token)  
    
        return ' '.join(new_doc)  

#  split dataset

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=10)


In [8]:
X_train_clean=list(map(text_cleaning,X_train))
X_test_clean=list(map(text_cleaning,X_test))

In [14]:
from  sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression,LinearRegression


tv=TfidfVectorizer()
X_new_matrix=tv.fit_transform(X_train_clean).toarray()
X_test_matrix=tv.transform(X_test_clean).toarray()
from sklearn.naive_bayes import GaussianNB,MultinomialNB
model1=GaussianNB()
model1.fit(X_new_matrix,y_train)

print("train sore",model1.score(X_new_matrix,y_train))
print("test sore",model1.score(X_test_matrix,y_test))


train sore 0.9298875329026083
test sore 0.8786791098348887


In [10]:
cv=CountVectorizer()
new_train=cv.fit_transform(X_train_clean)
new_test=cv.transform(X_test_clean)
model=LogisticRegression()
model.fit(new_train,y_train)
print("train sore",model.score(new_train,y_train))
print("test sore",model.score(new_test,y_test))

train sore 0.9954534577650156
test sore 0.9777458722182341


In [11]:
import joblib

In [15]:
joblib.dump(model1,"Spam_prediction_model")

['Spam_prediction_model']

In [17]:
model=joblib.load("Spam_prediction_model")

#  testing part


In [18]:
test_cor=["WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.",
"give your account details to get the offer","you won a free trip of world tour","Claim Your $1,000 Gift Card Now!"]		
test_new=list(map(text_cleaning,test_cor))
clean_test=tv.transform(test_new).toarray()
print(model.predict(clean_test))
print(model.predict_proba(clean_test))

['spam' 'spam' 'spam' 'spam']
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [19]:
test_file=pd.read_csv("d:/dataset/dataset/sentiment/test_data_spam.txt",sep=('\t'),header=None,names=['text'])
test_file1=test_file['text']
test_new=list(map(text_cleaning,test_file1))
clean_test=tv.transform(test_new).toarray()
predictions = model.predict(clean_test)
pred1=[]
for x,y in zip(test_file1,predictions):
    pred1.append((x,y))
pred1


[('you won a free trip of world tour', 'spam'),
 ('give your account details to get the offer', 'spam'),
 ("Subject: Urgent! You've Won a $1,000 Gift Card!", 'spam'),
 ('Claim Your $1,000 Gift Card Now!', 'spam'),
 ('hello sir,how are you?', 'spam'),
 ('congratulation,you got selected in final round of job process', 'ham'),
 ('you got selected as a lucky winner', 'spam')]